# MERGE DATASET - API DATA

### Libreries used

In [2]:
import pandas as pd
import re
from sqlalchemy import create_engine
import json
import sys
import os
import ast

## Connection to database

In [3]:
sys.path.append('../config/')
from dbconfig import configuration

config = configuration('../config/database.ini')
db_url = f"postgresql+psycopg2://{config['user']}:{config['password']}@{config['host']}/{config['database']}"
engine = create_engine(db_url)

try:
    conn = engine.connect()
    print("Database connection successful.")
except Exception as e:
    print("Database connection failed:", e)
    sys.exit(1)


Reading configuration from ../config/database.ini
Database connection failed: (psycopg2.OperationalError) connection to server at "host.docker.internal" (11.11.21.138), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


AttributeError: 'tuple' object has no attribute 'tb_frame'

Next, this part will be for the practical use of the dags and the follow-up of the steps.

### Load the CSV data into a DataFrame

In [4]:
metacritic_data_path = '../data/metacritic_data.csv'
try:
    metacritic_data = pd.read_csv(metacritic_data_path)
    print(f"CSV data loaded successfully with {len(metacritic_data)} records.")
except Exception as e:
    print("Failed to load CSV data:", e)
    sys.exit(1)

CSV data loaded successfully with 14055 records.


## API DATA

#### List of CSV files

In [5]:
api_data_path = '../data/'
csv_files = [os.path.join(api_data_path, f'api_data_{i}.csv') for i in range(1, 14)]

#### Store DataFrames

In [6]:
dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
        print(f"Loaded {file} successfully with {len(df)} records.")
    except Exception as e:
        print(f"Failed to load {file}:", e)
        sys.exit(1)

Loaded ../data/api_data_1.csv successfully with 1000 records.
Loaded ../data/api_data_2.csv successfully with 1000 records.
Loaded ../data/api_data_3.csv successfully with 900 records.
Loaded ../data/api_data_4.csv successfully with 240 records.
Loaded ../data/api_data_5.csv successfully with 1000 records.
Loaded ../data/api_data_6.csv successfully with 1000 records.
Loaded ../data/api_data_7.csv successfully with 1000 records.
Loaded ../data/api_data_8.csv successfully with 1000 records.
Loaded ../data/api_data_9.csv successfully with 1000 records.
Loaded ../data/api_data_10.csv successfully with 1000 records.
Loaded ../data/api_data_11.csv successfully with 1000 records.
Loaded ../data/api_data_12.csv successfully with 1000 records.
Loaded ../data/api_data_13.csv successfully with 1000 records.


#### Concatenate all DataFrames into a single DataFrame

In [7]:
combined_api_data = pd.concat(dataframes, ignore_index=True)
print(f"Combined API data has {len(combined_api_data)} records.")

Combined API data has 12140 records.


In [12]:
try:
    combined_api_data.to_sql('api_data', conn, if_exists='replace', index=False)
    print("Data loaded into the PostgreSQL database successfully.")
except Exception as e:
    print("Failed to load data into the PostgreSQL database:", e)
    sys.exit(1)

Data loaded into the PostgreSQL database successfully.


## General description

In [445]:
combined_api_data.head()

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,...,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating
0,9907,pictopix,Pictopix,2017-01-05,False,https://media.rawg.io/media/screenshots/ff0/ff...,3.96,4,"[{'id': 4, 'title': 'recommended', 'count': 20...",24,...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 40, 'name': 'Casual', 'slug': 'casual'...","[{'id': 10805, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
1,4119,chronovolt,Chronovolt,2012-11-21,False,https://media.rawg.io/media/games/8dc/8dc50d62...,2.55,3,"[{'id': 3, 'title': 'meh', 'count': 11, 'perce...",20,...,0f0f0f,"[{'platform': {'id': 19, 'name': 'PS Vita', 's...","[{'platform': {'id': 2, 'name': 'PlayStation',...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 4440, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 37796, 'name': 'exclusive', 'slug': 'e...","{'id': 1, 'name': 'Everyone', 'slug': 'everyone'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
2,3935,breakquest-extra-evolution,BreakQuest: Extra Evolution,2012-10-16,False,https://media.rawg.io/media/screenshots/4e3/4e...,2.60,3,"[{'id': 3, 'title': 'meh', 'count': 10, 'perce...",20,...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 2, 'name': 'PlayStation',...","[{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...","[{'id': 4253, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 114, 'name': 'Physics', 'slug': 'physi...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
3,1991,killallzombies-2,#KILLALLZOMBIES,2014-10-28,False,https://media.rawg.io/media/screenshots/675/67...,2.36,1,"[{'id': 1, 'title': 'skip', 'count': 5, 'perce...",11,...,0f0f0f,"[{'platform': {'id': 1, 'name': 'Xbox One', 's...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 2061, 'store': {'id': 2, 'name': 'Xbox...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
4,979,tethered,Tethered,2016-10-25,False,https://media.rawg.io/media/screenshots/1e6/1e...,2.75,4,"[{'id': 4, 'title': 'recommended', 'count': 5,...",12,...,0f0f0f,"[{'platform': {'id': 18, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 3, 'name': 'Adventure', 'slug': 'adven...","[{'id': 1002, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN


In [446]:
combined_api_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12140 entries, 0 to 12139
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  12140 non-null  int64  
 1   slug                12140 non-null  object 
 2   name                12140 non-null  object 
 3   released            11881 non-null  object 
 4   tba                 12140 non-null  bool   
 5   background_image    12120 non-null  object 
 6   rating              12140 non-null  float64
 7   rating_top          12140 non-null  int64  
 8   ratings             12140 non-null  object 
 9   ratings_count       12140 non-null  int64  
 10  reviews_text_count  12140 non-null  int64  
 11  added               12140 non-null  int64  
 12  added_by_status     12140 non-null  object 
 13  metacritic          4628 non-null   float64
 14  playtime            12140 non-null  int64  
 15  suggestions_count   12140 non-null  int64  
 16  upda

In [447]:
metacritic_data.head()

,title,released,developer,publisher,genres,rating,user score,ratings_count,platforms
0,ZIGGURAT,2012-02-17,ACTION BUTTON ENTERTAINMENT,FRESHUU INC.,ACTION,NaN,6.9,14,IOS (IPHONE/IPAD)
1,4X4 EVO 2,2001-11-15,TERMINAL REALITY,GATHERING,AUTO RACING SIM,RATED E FOR EVERYONE,0.0,0,XBOX
2,MOTOGP 2,2002-01-22,NAMCO,NAMCO,AUTO RACING SIM,RATED E FOR EVERYONE,5.8,0,PLAYSTATION 2
3,GOTHIC 3,2006-11-14,PIRANHA BYTES,ASPYR,WESTERN RPG,RATED T FOR TEEN,7.5,832,PC
4,SIEGE SURVIVAL GLORIA VICTIS,2021-05-18,FISHTANKSTUDIO,BLACK EYE GAMES,RPG,NaN,6.5,10,PC


In [448]:
metacritic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14055 entries, 0 to 14054
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          14034 non-null  object 
 1   released       13991 non-null  object 
 2   developer      13917 non-null  object 
 3   publisher      13917 non-null  object 
 4   genres         14034 non-null  object 
 5   rating         11005 non-null  object 
 6   user score     14055 non-null  float64
 7   ratings_count  14055 non-null  int64  
 8   platforms      14026 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 988.4+ KB


## Transformations and merge

Function to extract platforms names

In [8]:
def extract_classification_names(json_str):
    # Check if the input is actually a string
    if isinstance(json_str, str):
        try:
            # Assuming the JSON strings are formatted with double quotes
            classification_data = json.loads(json_str.replace("'", '"'))
            # If the structure contains a list of platforms, we join them with a comma
            if isinstance(classification_data, list):
                classification_names = [classification['name'] for classification in classification_data]
                return ', '.join(classification_names)
            # If it's a dictionary and 'name' is the key for platform names
            elif 'name' in classification_data:
                return classification_data['name']
            else:
                return None
        except json.JSONDecodeError:
            return None
    else:
        return None

combined_api_data['rating'] = combined_api_data['esrb_rating'].apply(extract_classification_names)

# Check the transformation results
print(combined_api_data[['rating']].head())

         rating
0          None
1      Everyone
2  Everyone 10+
3          None
4  Everyone 10+


In [9]:
def extract_first_genre_name(json_str):
    if isinstance(json_str, str):
        try:
            # Correctly parse the JSON assuming single quotes in string representation
            genre_data = json.loads(json_str.replace("'", '"'))
            # Check if the data structure is a dictionary (single genre) or a list (multiple genres)
            if isinstance(genre_data, dict):
                # Return the name directly if it's a single genre
                return genre_data['name']
            elif isinstance(genre_data, list) and len(genre_data) > 0:
                # Return the name of the first genre if there are multiple
                return genre_data[0]['name']
            else:
                return None
        except json.JSONDecodeError:
            return None
    else:
        return None

# Apply the modified function to the 'genres' column
combined_api_data['genres'] = combined_api_data['genres'].apply(extract_first_genre_name)

# Display the result to verify
print(combined_api_data[['genres']])

          genres
0         Casual
1         Action
2         Arcade
3         Action
4      Adventure
...          ...
12135     Racing
12136     Action
12137  Adventure
12138        RPG
12139  Adventure

[12140 rows x 1 columns]


In [10]:
# Function to search for the word 'name' and extract the first platform name
def extract_first_platform_name_from_string(json_str, index):
    if not json_str or json_str.strip() in ['platforms', '']:
        print(f"Row {index}: Empty or placeholder string")
        return None
    
    # Check if the string is a plain text value
    if json_str.isalpha():
        print(f"Row {index}: Plain text value: {json_str}")
        return json_str
    
    try:
        # Replace single quotes with double quotes to form valid JSON
        json_str = json_str.replace("'", '"')
        
        # Find the platform names using regular expressions
        matches = re.findall(r'"name":\s*"([^"]+)"', json_str)
        if matches:
            return matches[0]  # Return the first match
    except Exception as e:
        print(f"Row {index}: Error processing string: {e} - {json_str}")
        return None


# Apply the function to the DataFrame
combined_api_data['platforms'] = combined_api_data.apply(lambda row: extract_first_platform_name_from_string(row['platforms'], row.name), axis=1)

# Display the DataFrame
print(combined_api_data['platforms'])

0                   PC
1              PS Vita
2        PlayStation 3
3             Xbox One
4        PlayStation 4
             ...      
12135               PC
12136               PC
12137            macOS
12138               PC
12139               PC
Name: platforms, Length: 12140, dtype: object


Genres in PostgreSQL are comma-separated strings, we will join API genres lists

In [452]:
# Select relevant columns and make a copy to avoid SettingWithCopyWarning
api_relevant_columns = [
    'name', 'released', 'rating', 'ratings_count', 'metacritic', 'genres', 'platforms'
]
api_data_relevant = combined_api_data[api_relevant_columns].copy()


# Standardize titles and release dates to ensure consistent formatting
api_data_relevant['name'] = api_data_relevant['name'].str.strip().str.upper()
api_data_relevant['released'] = pd.to_datetime(api_data_relevant['released']).dt.date



In [453]:
api_data_relevant['name'] = api_data_relevant['name'].str.lower() \
                         .str.replace(r"\(.*\)", "", regex=True) \
                         .str.replace(r"[-:,$#.//'\[\]()]", "", regex=True)

In [454]:
# Rename columns to match PostgreSQL data
api_data_relevant.rename(columns={
    'name': 'title',
    'rating': 'rating',
    'metacritic': 'metacritic_score',
    'ratings_count': 'ratings_count'
}, inplace=True)


In [455]:
rating_mapping = {
    "Adults Only": "RATED AO FOR ADULTS ONLY",
    "Everyone": "RATED E FOR EVERYONE",
    "Everyone +10": "RATED E +10 FOR EVERYONE +10",
    "Mature": "RATED M FOR MATURE",
    "Rating Pending": "RATED RP FOR RATE PENDING",
    "Teen": "RATED T FOR TEEN"
}
# Define the mapping dictionary
rating_mapping = {
    "Adults Only": "RATED AO FOR ADULTS ONLY",
    "Everyone": "RATED E FOR EVERYONE",
    "Everyone +10": "RATED E +10 FOR EVERYONE +10",
    "Mature": "RATED M FOR MATURE",
    "Rating Pending": "RATED RP FOR RATE PENDING",
    "Teen": "RATED T FOR TEEN"
}

# Apply the mapping to the 'rating' column in api_data_relevant
api_data_relevant['rating'] = api_data_relevant['rating'].map(rating_mapping)

Rename columns to match PostgreSQL data

In [456]:
# Standardize titles and release dates in db_data to ensure consistent formatting
metacritic_data['title'] = metacritic_data['title'].str.strip().str.upper()
metacritic_data['released'] = pd.to_datetime(metacritic_data['released']).dt.date

# Debug: Display the column names of both DataFrames
print("API Data Relevant Columns:", api_data_relevant.columns.tolist())
api_data_relevant.head()


API Data Relevant Columns: ['title', 'released', 'rating', 'ratings_count', 'metacritic_score', 'genres', 'platforms']


,title,released,rating,ratings_count,metacritic_score,genres,platforms
0,pictopix,2017-01-05,NaN,24,NaN,Casual,PC
1,chronovolt,2012-11-21,RATED E FOR EVERYONE,20,44.0,Action,PS Vita
2,breakquest extra evolution,2012-10-16,NaN,20,NaN,Arcade,PlayStation 3
3,killallzombies,2014-10-28,NaN,11,NaN,Action,Xbox One
4,tethered,2016-10-25,NaN,12,NaN,Adventure,PlayStation 4


In [457]:
api_data_relevant.to_csv('../data/api_data3.csv', index=False)
print("Combined API data saved to 'api_data3.csv'")

Combined API data saved to 'api_data3.csv'


In [458]:
print("Database Data Columns:", metacritic_data.columns.tolist())
metacritic_data.head()

Database Data Columns: ['title', 'released', 'developer', 'publisher', 'genres', 'rating', 'user score', 'ratings_count', 'platforms']


,title,released,developer,publisher,genres,rating,user score,ratings_count,platforms
0,ZIGGURAT,2012-02-17,ACTION BUTTON ENTERTAINMENT,FRESHUU INC.,ACTION,NaN,6.9,14,IOS (IPHONE/IPAD)
1,4X4 EVO 2,2001-11-15,TERMINAL REALITY,GATHERING,AUTO RACING SIM,RATED E FOR EVERYONE,0.0,0,XBOX
2,MOTOGP 2,2002-01-22,NAMCO,NAMCO,AUTO RACING SIM,RATED E FOR EVERYONE,5.8,0,PLAYSTATION 2
3,GOTHIC 3,2006-11-14,PIRANHA BYTES,ASPYR,WESTERN RPG,RATED T FOR TEEN,7.5,832,PC
4,SIEGE SURVIVAL GLORIA VICTIS,2021-05-18,FISHTANKSTUDIO,BLACK EYE GAMES,RPG,NaN,6.5,10,PC


In [459]:
metacritic_data.to_csv('../data/metacritic_data2.csv', index=False)
print("Combined API data saved to 'metacritic_data2.csv'")

Combined API data saved to 'metacritic_data2.csv'


In [460]:
# Ensure 'genres' and 'platforms' columns exist in both DataFrames
if 'genres' not in api_data_relevant.columns or 'genres' not in metacritic_data.columns:
    print("Error: 'genres' column is missing in one of the DataFrames.")
    sys.exit(1)

if 'platforms' not in api_data_relevant.columns or 'platforms' not in metacritic_data.columns:
    print("Error: 'platforms' column is missing in one of the DataFrames.")
    sys.exit(1)

In [461]:
api_data_relevant = api_data_relevant.map(lambda x: x.upper() if isinstance(x, str) else x)

In [462]:
# Merge the combined API data with the PostgreSQL data
merged_data = pd.merge(api_data_relevant, metacritic_data, on=['title', 'released', 'genres', 'platforms', 'rating', 'ratings_count'], how='outer')

# Display rows with NaN values to understand the mismatches
nan_rows = merged_data[merged_data.isna().any(axis=1)]
print("Rows with NaN values:")
print(nan_rows)

Rows with NaN values:
                                       title    released  \
0      "GLOW BALL"  THE BILLIARD PUZZLE GAME  2015-07-20   
1                              'SPLOSION MAN  2009-07-22   
2                      0 DAY ATTACK ON EARTH  2009-12-23   
3                                007 LEGENDS  2012-10-16   
4                                007 LEGENDS  2012-10-16   
...                                      ...         ...   
26190                                    NaN         NaT   
26191                                    NaN         NaT   
26192                                    NaN         NaT   
26193                                    NaN         NaT   
26194                                    NaN         NaT   

                             rating  ratings_count  metacritic_score  \
0                               NaN              4               NaN   
1      RATED E +10 FOR EVERYONE +10             91               NaN   
2      RATED E +10 FOR EVERYONE +10      

In [463]:
# Display merged data
print("Merged Data:")
merged_data.head()

Merged Data:


,title,released,rating,ratings_count,metacritic_score,genres,platforms,developer,publisher,user score
0,"""GLOW BALL"" THE BILLIARD PUZZLE GAME",2015-07-20,NaN,4,NaN,STRATEGY,PC,NaN,NaN,NaN
1,'SPLOSION MAN,2009-07-22,RATED E +10 FOR EVERYONE +10,91,NaN,2D PLATFORMER,XBOX 360,TWISTED PIXEL GAMES,TWISTED PIXEL GAMES,7.3
2,0 DAY ATTACK ON EARTH,2009-12-23,RATED E +10 FOR EVERYONE +10,8,NaN,THIRD PERSON SHOOTER,XBOX 360,GULTI,SQUARE ENIX,5.9
3,007 LEGENDS,2012-10-16,RATED T FOR TEEN,44,37.0,ACTION,WII U,NaN,NaN,NaN
4,007 LEGENDS,2012-10-16,RATED T FOR TEEN,84,NaN,FPS,XBOX 360,EUROCOM,ACTIVISION,4.5


In [464]:
merged_data.drop(columns=['metacritic_score', 'user score', 'developer', 'publisher'], inplace=True)

In [465]:
merged_data.head()

,title,released,rating,ratings_count,genres,platforms
0,"""GLOW BALL"" THE BILLIARD PUZZLE GAME",2015-07-20,NaN,4,STRATEGY,PC
1,'SPLOSION MAN,2009-07-22,RATED E +10 FOR EVERYONE +10,91,2D PLATFORMER,XBOX 360
2,0 DAY ATTACK ON EARTH,2009-12-23,RATED E +10 FOR EVERYONE +10,8,THIRD PERSON SHOOTER,XBOX 360
3,007 LEGENDS,2012-10-16,RATED T FOR TEEN,44,ACTION,WII U
4,007 LEGENDS,2012-10-16,RATED T FOR TEEN,84,FPS,XBOX 360


In [466]:
merged_data.to_csv('../data/merged_data.csv', index=False)
print("Merged data saved to 'merged_data.csv'")

Merged data saved to 'merged_data.csv'


In [467]:
# Display rows with NaN values to understand the mismatches
nan_rows = merged_data[merged_data.isna().any(axis=1)]
print("Rows with NaN values:")
print(nan_rows)

Rows with NaN values:
                                       title    released rating  \
0      "GLOW BALL"  THE BILLIARD PUZZLE GAME  2015-07-20    NaN   
10                                 0RBITALIS  2015-05-28    NaN   
11                                 0RBITALIS  2015-05-28    NaN   
13              1 MOMENT OF TIME SILENTVILLE  2016-07-22    NaN   
14                                    1 RIDE  2016-07-28    NaN   
...                                      ...         ...    ...   
26190                                    NaN         NaT    NaN   
26191                                    NaN         NaT    NaN   
26192                                    NaN         NaT    NaN   
26193                                    NaN         NaT    NaN   
26194                                    NaN         NaT    NaN   

       ratings_count         genres platforms  
0                  4       STRATEGY        PC  
10                 5  ACTION PUZZLE        PC  
11                 7     SIMU

In [468]:
# Drop rows with NaN values
cleaned_merged_data = merged_data.dropna()

# Display cleaned merged data
print("Cleaned Merged Data:")
print(cleaned_merged_data.head())

Cleaned Merged Data:
                   title    released                        rating  \
1          'SPLOSION MAN  2009-07-22  RATED E +10 FOR EVERYONE +10   
2  0 DAY ATTACK ON EARTH  2009-12-23  RATED E +10 FOR EVERYONE +10   
3            007 LEGENDS  2012-10-16              RATED T FOR TEEN   
4            007 LEGENDS  2012-10-16              RATED T FOR TEEN   
5          007 NIGHTFIRE  2002-11-18              RATED T FOR TEEN   

   ratings_count                genres platforms  
1             91         2D PLATFORMER  XBOX 360  
2              8  THIRD PERSON SHOOTER  XBOX 360  
3             44                ACTION     WII U  
4             84                   FPS  XBOX 360  
5             20                   FPS  GAMECUBE  


In [469]:
# Drop rows with NaN values
cleaned_merged_data = merged_data.dropna()

In [470]:
cleaned_merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14707 entries, 1 to 26152
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          14707 non-null  object
 1   released       14707 non-null  object
 2   rating         14707 non-null  object
 3   ratings_count  14707 non-null  int64 
 4   genres         14707 non-null  object
 5   platforms      14707 non-null  object
dtypes: int64(1), object(5)
memory usage: 804.3+ KB
